In [3]:
import numpy as np
import os
from json_tricks import dump, load
from pydub import AudioSegment, effects
import librosa
import noisereduce as nr
import tensorflow as tf
from keras import models
from keras import layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
sample_lengths = []
folder_path = 'Dataset'

for subdir, dirs, files in os.walk(folder_path):
  for file in files: 
    x, sr = librosa.load(path = os.path.join(subdir,file), sr = None)
    xt, index = librosa.effects.trim(x, top_db=30)
     
    sample_lengths.append(len(xt))

print('Maximum sample length:', np.max(sample_lengths))  

Maximum sample length: 204288


In [4]:
def find_emotion(name):
    if 'neutral' in name: return "01"
    elif 'happy' in name: return "03"
    elif 'sad' in name: return "04"
    elif 'angry' in name: return "05"
    elif 'fear' in name: return "06"
    elif 'disgust' in name: return "07"
    elif 'ps' in name: return "08"
    else: return "-1"

def emotion_fix(e_num):
    if e_num == "01": return 0
    elif e_num == "02": return 1
    elif e_num == "03": return 2
    elif e_num == "04": return 3
    elif e_num == "05": return 4
    elif e_num == "06": return 5
    elif e_num == "07": return 6
    else: return 7


In [19]:
import time
tic = time.perf_counter()

# Initialize data lists
rms = []
zcr = []
mfcc = []
emotions = []

# Initialize variables
total_length = 204288 # desired frame length for all audio samples.
frame_length = 2048
hop_length = 512

folder_path = 'Dataset' 

for subdir, dirs, files in os.walk(folder_path):
  for file in files: 

    # Fetch the sample rate.
      _, sr = librosa.load(path = os.path.join(subdir,file), sr = None) # sr (the sample rate) is used for librosa's MFCCs. '_' is irrelevant.
    # Load the audio file.
      rawsound = AudioSegment.from_file(os.path.join(subdir,file)) 
    # Normalize the audio to +5.0 dBFS.
      normalizedsound = effects.normalize(rawsound, headroom = 0) 
    # Transform the normalized audio to np.array of samples.
      normal_x = np.array(normalizedsound.get_array_of_samples(), dtype = 'float32')
    # Trim silence from the beginning and the end.
      xt, index = librosa.effects.trim(normal_x, top_db=30)
    # print(file,"\t", len(xt), "\t", rawsound.dBFS, "\t", normalizedsound.dBFS) #--QA purposes if needed-- 
    # Pad for duration equalization.
      if len(xt) > total_length:
        xt = xt[:total_length]
      padded_x = np.pad(xt, (0, total_length-len(xt)), 'constant')
    # Noise reduction.
      final_x = nr.reduce_noise(padded_x, sr=sr) #updated 03/03/22
       
   # Features extraction 
      f1 = librosa.feature.rms(y=final_x, frame_length=frame_length, hop_length=hop_length) # Energy - Root Mean Square   
      f2 = librosa.feature.zero_crossing_rate(final_x , frame_length=frame_length, hop_length=hop_length, center=True) # ZCR      
      f3 = librosa.feature.mfcc(y=final_x, sr=sr, n_mfcc=13, hop_length = hop_length) # MFCC
      
   # Emotion extraction from the different databases
      if (find_emotion(file) != "-1"): #TESS database validation
            name = find_emotion(file)
      else:                              #RAVDESS database validation
            name = file[6:8]                      

   # Filling the data lists  
      rms.append(f1)
      zcr.append(f2)
      mfcc.append(f3)
      emotions.append(emotion_fix(name)) 

toc = time.perf_counter()
print(f"Running time: {(toc - tic)/60:0.4f} minutes")

Running time: 2.5009 minutes


In [20]:
f_rms = np.asarray(rms).astype('float32')
f_rms = np.swapaxes(f_rms,1,2)
f_zcr = np.asarray(zcr).astype('float32')
f_zcr = np.swapaxes(f_zcr,1,2)
f_mfccs = np.asarray(mfcc).astype('float32')
f_mfccs = np.swapaxes(f_mfccs,1,2)

print('ZCR shape:',f_zcr.shape)
print('RMS shape:',f_rms.shape)
print('MFCCs shape:',f_mfccs.shape)


ZCR shape: (1200, 400, 1)
RMS shape: (1200, 400, 1)
MFCCs shape: (1200, 400, 13)


In [21]:
# Concatenating all features to 'X' variable.
X = np.concatenate((f_zcr, f_rms, f_mfccs), axis=2)

# Preparing 'Y' as a 2D shaped variable.
Y = np.asarray(emotions).astype('int8')
Y = np.expand_dims(Y, axis=1)

In [22]:
x_data = X.tolist() 
x_path = 'Dataset/X_datanew.json' # FILE SAVE PATH
dump(obj = x_data, fp = x_path)

y_data = Y.tolist() 
y_path = 'Dataset/Y_datanew.json' # FILE SAVE PATH
dump(obj = y_data, fp = y_path)

'[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]

In [23]:
x_path = 'Dataset/X_datanew.json' # FILE LOAD PATH
X = load(x_path)
X = np.asarray(X, dtype = 'float32')

y_path = 'Dataset/Y_datanew.json' # FILE LOAD PATH
Y = load(y_path)
Y = np.asarray(Y, dtype = 'int8')

In [26]:
# Split to train, validation, and test sets.
from sklearn.model_selection import train_test_split
x_train, x_tosplit, y_train, y_tosplit = train_test_split(X, Y, test_size = 0.125, random_state = 1)
x_val, x_test, y_val, y_test = train_test_split(x_tosplit, y_tosplit, test_size = 0.304, random_state = 1)

#'One-hot' vectors for Y: emotion classification
y_train_class = tf.keras.utils.to_categorical(y_train, 8).astype('int8')
y_val_class = tf.keras.utils.to_categorical(y_val, 8).astype('int8')

In [27]:
print(np.shape(x_train))
print(np.shape(x_val))
print(np.shape(x_test))

(1050, 400, 15)
(104, 400, 15)
(46, 400, 15)


In [28]:
file_path = 'x_test_data.json'
dump(obj = x_test, fp = file_path)

file_path = 'y_test_data.json'
dump(obj = y_test, fp = file_path)

'{"__ndarray__": [[5], [5], [6], [7], [6], [3], [4], [4], [3], [1], [1], [2], [6], [2], [5], [0], [6], [6], [7], [1], [6], [4], [2], [3], [1], [3], [4], [7], [7], [6], [3], [6], [7], [2], [7], [4], [2], [6], [3], [0], [6], [1], [2], [2], [2], [4]], "dtype": "int8", "shape": [46, 1], "Corder": true}'

In [32]:
model = models.Sequential()
model.add(layers.LSTM(64, return_sequences = True, input_shape=(X.shape[1:3])))
model.add(layers.LSTM(64))
model.add(layers.Dense(8, activation = 'softmax'))
print(model.summary())

batch_size = 23

# Callbacks functions
checkpoint_path = 'models/best_weights.keras'

#-> Save the best weights
mcp_save = callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True,
                           monitor='val_categorical_accuracy',
                           mode='max')
#-> Reduce learning rate after 100 epoches without improvement.
rlrop = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', 
                                    factor=0.1, patience=100)
                             
# Compile & train   
model.compile(loss='categorical_crossentropy', 
                optimizer='RMSProp', 
                metrics=['categorical_accuracy'])

history = model.fit(x_train, y_train_class, 
                      epochs=340, batch_size = batch_size, 
                      validation_data = (x_val, y_val_class), 
                      callbacks = [mcp_save, rlrop])
# Define the best weights to the model.
model.load_weights(checkpoint_path)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 400, 64)        │        20,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,024 (211.03 KB)

 Trainable params: 54,024 (211.03 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/340
46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 137ms/step - categorical_accuracy: 0.1288 - loss: 2.0878 - val_categorical_accuracy: 0.0865 - val_loss: 2.0615 - learning_rate: 0.0010
Epoch 2/340
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step - categorical_accuracy: 0.1207 - loss: 2.0754 - val_categorical_accuracy: 0.1058 - val_loss: 2.0686 - learning_rate: 0.0010
Epoch 3/340
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 126ms/step - categorical_accuracy: 0.1538 - loss: 2.0609 - val_categorical_accuracy: 0.0865 - val_loss: 2.0708 - learning_rate: 0.0010
Epoch 4/340
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 128ms/step - categorical_accuracy: 0.1365 - loss: 2.0710 - val_categorical_accuracy: 0.0865 - val_loss: 2.0474 - learning_rate: 0.0010
Epoch 5/340
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - categorical_accuracy: 0.1243 - loss: 2.0651 - val_categorical_accuracy: 0.1058 - val_loss: 2.0660 - learning_rate: 0.0010
Epoch 6/340
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 135ms/step - categorical_accuracy: 0.1214 - loss: 2.0710 - val_categ

KeyboardInterrupt: 